In [51]:
# import all the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict,OrderedDict
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_distances
from operator import itemgetter
from scipy.spatial.distance import cosine
import json
import pickle

In [52]:
# reading data from dataset
df = pd.read_csv('temp/db1.csv')
%store df
# breif about the columns in the dataset
df.head()

Stored 'df' (DataFrame)


,id,phoneNo,location,occupation,certification,age,gender,type,availabilityPreference,availability,minimumWage,experience,clientsAttended,aadharCard,doorstepService,references,liscenced,shoppingliscence
0,1,9326523642,Ambegaon,Accountant,True,26,Female,Full Time,Afternoon,Whole Day,9500,8,49,65660562714,no,False,False,True
1,2,7029738484,Ambegaon,Brick mason,True,36,Male,Part Time,Morning,Morning,4000,4,25,394728990924,no,False,False,True
2,3,8386070476,Ambegaon,Carpenter,True,31,Male,Part Time,Evening,Evening,5500,8,42,777556713881,yes,True,False,False
3,4,8208265762,Ambegaon,Cashier,False,20,Female,Part Time,Evening,Evening,2500,2,13,938143072796,no,True,False,True
4,5,9127458171,Ambegaon,Childcare Worker,True,33,Female,Full Time,Evening,Whole Day,2000,3,28,112614367501,yes,True,False,True


In [53]:
df.drop(['phoneNo','id','availabilityPreference','aadharCard','liscenced','shoppingliscence'],axis=1,inplace=True)
df.dropna(inplace=True)   
    # dummy classes
df.head()

,location,occupation,certification,age,gender,type,availability,minimumWage,experience,clientsAttended,doorstepService,references
0,Ambegaon,Accountant,True,26,Female,Full Time,Whole Day,9500,8,49,no,False
1,Ambegaon,Brick mason,True,36,Male,Part Time,Morning,4000,4,25,no,False
2,Ambegaon,Carpenter,True,31,Male,Part Time,Evening,5500,8,42,yes,True
3,Ambegaon,Cashier,False,20,Female,Part Time,Evening,2500,2,13,no,True
4,Ambegaon,Childcare Worker,True,33,Female,Full Time,Whole Day,2000,3,28,yes,True


In [54]:
def classes_maker():
    for i in df.columns:
        le = preprocessing.LabelEncoder()
        le.fit(df[i])
        df[i] = le.transform(df[i])
classes_maker()
df.head()

,location,occupation,certification,age,gender,type,availability,minimumWage,experience,clientsAttended,doorstepService,references
0,0,0,1,8,0,0,3,15,7,44,0,0
1,0,1,1,18,1,1,2,4,3,20,0,0
2,0,2,1,13,1,1,1,7,7,37,1,1
3,0,3,0,2,0,1,1,1,1,8,0,1
4,0,4,1,15,0,0,3,0,2,23,1,1


In [55]:
#intermediate
occupations=defaultdict(list)
def all_occupations_in_a_location():
    for index,row in df.iterrows():
        occupations[row['location']].append((index,row['occupation']))
        
    for key, values in occupations.items():
        t_set = set(occupations.get(key))
        occupations[key] = list(t_set)
all_occupations_in_a_location()

In [56]:
def visualizing():
    pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(19, 19), diagonal='kde')
    plt.show()

In [57]:
def storage():
    %store occupations
    %store df
storage()

Stored 'occupations' (defaultdict)
Stored 'df' (DataFrame)


In [58]:
# with open('occupations.txt','w') as f:
#     f.write(str(pickle.dumps(occupations)))
# print("Occupation dict has been pickled.")

In [59]:
# dist = np.ndarray(shape=(np.shape(df)[0],np.shape(df)[0]))
cosdist = np.ndarray(shape=(np.shape(df)[0],np.shape(df)[0]))
eucdist = np.ndarray(shape=(np.shape(df)[0],np.shape(df)[0]))

def cos_func(index1,index2):
    # will calcuate the cosine distance between 2 workers and return to helper
    #print(index1,index2)
    return cosine_distances(df.loc[index1].values.reshape(1, -1),df.loc[index2].values.reshape(1, -1))

def euc_dist(index1,index2):
    return euclidean_distances(df.loc[index1].values.reshape(1, -1),df.loc[index2].values.reshape(1, -1))

def cos_func(index1,index2):
    return cosine_distances(df.loc[index1].values.reshape(1, -1),df.loc[index2].values.reshape(1, -1))

def cos_helper(indexes):
    for i in indexes:
        for j in indexes:
            if i != j:
                cosdist[i][j] = cos_func(i,j)
                eucdist[i][j] = euc_dist(i,j)

def commonities():
        for k, v in occupations.items(): 
            type(occupations[k]) # list type
            temp_list = occupations[k]
            indexes = np.array([])
            temp_list.sort(key=itemgetter(1)) # sort list according to occupation
            
            for i in range(len(occupations[k])):
                if i+1 < len(occupations[k]):
                    if temp_list[i][1] == temp_list[i+1][1]:
                        np.append(indexes,temp_list[i][0])
                        np.append(indexes,temp_list[i+1][0])
                        
            cos_helper(list(set(indexes)))
        #return list(set(indexes))
                
commonities()
print('Distances calculated')


Distances calculated


In [60]:
eucdist
# plt.plot(dist)
# plt.show()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [61]:
cosdist

array([[  4.67851852e-310,   6.92120667e-310,   4.67851852e-310, ...,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000]])

In [62]:
# cosine_distances(df.loc[11010].values.reshape(1,-1),df.loc[4003].values.reshape(1,-1))